In [59]:
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import re
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn.metrics.pairwise import cosine_similarity

In [60]:
nltk.download("stopwords")
stope_words_set = set(stopwords.words("english"))
from nltk.tokenize import word_tokenize

def remove_stop_words(summary):
    flag = 0
    summary_word_token = word_tokenize(summary)
    updated_summary = ""
    for w in summary_word_token:
        if w not in stope_words_set:
            if flag == 1:
                updated_summary = updated_summary + " " + w
            else:
                updated_summary = w
                flag = flag +1
    #print(updated_summary)
    return updated_summary

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aditi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [61]:
def input_encoding(url):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Referer': 'https://www.google.com/',
}

    response = requests.get(url, headers=headers , timeout=15)
    response.raise_for_status()

    sample_soup = BeautifulSoup(response.text, 'html.parser')
    
    encoded = []
    
    #Name
    sample_name = sample_soup.find('h1' ,{'class' : 'sc-d8941411-0 dxeMrU'})
    sample_name = sample_name.find('span' ,{'class' : 'hero__primary-text'}).text
    encoded.append(sample_name)
    print(sample_name)
    
    #Rating
    encoded.append(sample_soup.find('span',{'class':'sc-eb51e184-1 cxhhrI'}).text)
    print(sample_soup.find('span',{'class':'sc-eb51e184-1 cxhhrI'}).text)
    
    #Year
    year_duration = sample_soup.find('ul',{'class' : 'ipc-inline-list ipc-inline-list--show-dividers sc-d8941411-2 cdJsTz baseAlt'})
    year_duration = year_duration.find_all('li',{'class':'ipc-inline-list__item'})
    flag=0
    for item in year_duration:
        if flag == 0:
            sample_year=item.text
            encoded.append(sample_year)
            print(sample_year)
            flag = flag+1
            
    #Genre
    sample_genre=[]
    sample_genre_tag = sample_soup.find('div',{'class':'ipc-chip-list__scroller'})
    sample_genre_tag = sample_genre_tag.find_all('a',{'class':'ipc-chip ipc-chip--on-baseAlt'})
    for item in sample_genre_tag:
        g = item.find('span',{'class':'ipc-chip__text'}).text
        sample_genre.append(g)
    encoded.append(sample_genre[0])
    print(sample_genre[0])
    
    #Summary
    sample_intro = sample_soup.find('p' , {'class' : 'sc-2d37a7c7-3 dmZChI'})
    sample_intro = sample_intro.find('span' , {'class' : 'sc-2d37a7c7-0 caYjFF'}).text
    encoded.append(remove_stop_words(sample_intro))
    print(remove_stop_words(sample_intro))
    
    input_string = ""
    flag=0
    for item in encoded:
        if flag == 0:
            input_string = item
            flag = flag +1
        else:
            input_string = input_string + " " + item
            
    return input_string
            
def vectorization(corpus):
    vectorize = TfidfVectorizer(stop_words="english")
    vectorize_matrix = vectorize.fit_transform(corpus)
    return vectorize_matrix, vectorize
            
            
def find_similar_movies(vectorize_matrix, input_vector, movies_name, top_n=5):
    cosine_sim = cosine_similarity(input_vector, vectorize_matrix)
    similar_indices = cosine_sim.argsort().flatten()[-top_n:]
    similar_movies = [movies_name[idx] for idx in similar_indices]
    return similar_movies
        
    

In [62]:
with open(r'C:\Users\aditi\OneDrive\Desktop\PROJECTS\Movie-Recommendation-System\encoded_movies_info.txt', 'rb') as file:
    encoded_vector = pickle.load(file,encoding='utf-8')
with open(r'C:\Users\aditi\OneDrive\Desktop\PROJECTS\Movie-Recommendation-System\movies_name.txt', 'rb') as file:
    movies_name = pickle.load(file , encoding='utf-8') 

In [63]:
'''url = "https://www.imdb.com/title/tt8672856/"

sample_movie = input_encoding(url)
corpus = encoded_vector + [sample_movie]
vectorize_matrix, vectorize = vectorization(corpus)
input_vector = vectorize_matrix[-1]
vectorize_matrix = vectorize_matrix[:-1]
print(find_similar_movies(vectorize_matrix,input_vector,movies_name))'''

'url = "https://www.imdb.com/title/tt8672856/"\n\nsample_movie = input_encoding(url)\ncorpus = encoded_vector + [sample_movie]\nvectorize_matrix, vectorize = vectorization(corpus)\ninput_vector = vectorize_matrix[-1]\nvectorize_matrix = vectorize_matrix[:-1]\nprint(find_similar_movies(vectorize_matrix,input_vector,movies_name))'

In [64]:
df = pd.read_csv("movie_list.csv")

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Movie Name    248 non-null    object 
 1   Rating        248 non-null    float64
 2   Year          248 non-null    int64  
 3   Director      248 non-null    object 
 4   Writers       248 non-null    object 
 5   User Reviews  248 non-null    float64
 6   Genre         248 non-null    object 
 7   Summary       248 non-null    object 
 8   URL           248 non-null    object 
 9   Cast 1        248 non-null    object 
 10  Cast 2        248 non-null    object 
dtypes: float64(2), int64(1), object(8)
memory usage: 21.4+ KB


In [66]:
def get_imp_features(df):
    important_features =[]
    for i in df.index:
        important_features.append(df['Movie Name'][i] +' '+ str(df['Rating'][i]) + ' ' + str(df['Year'][i]) + ' ' + df['Genre'][i]+ ' ' + df['Summary'][i])
    return important_features

df['Important Features'] = get_imp_features(df)

In [67]:
url = "https://www.imdb.com/title/tt3001638/"

sample_movie = input_encoding(url)
corpus = df['Important Features'] + [sample_movie]
vectorize_matrix, vectorize = vectorization(corpus)
input_vector = vectorize_matrix[-1]
vectorize_matrix = vectorize_matrix[:-1]
print(find_similar_movies(vectorize_matrix,input_vector,movies_name))

Mary Kom
6.8
2014
Action
A chronicle life Indian boxer Mary Kom , went several hardships audaciously accomplishing ultimate dream .
['Charulata', 'Bombay', 'Veer-Zaara', 'Premam', 'Bommarillu']
